In [1]:
import numpy as np
import pandas as pd
import json
import os
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error
from datetime import datetime
%matplotlib inline

In [2]:
PATH_TO_DATA = '../../data/arktur_medium/'
!PATH_TO_DATA=../../data/arktur_medium/

INP_TRAIN = "train.json"
INP_TEST  = "test.json"
!INP_TRAIN=train.json
!INP_TEST=test.json

OUT_TRAIN = "train.vw"
OUT_PART_TRAIN = "part_train.vw"
OUT_PART_VALID = "part_valid.vw"
OUT_TEST = "test.vw"
!OUT_TRAIN=train.vw
!OUT_PART_TRAIN=part_train.vw
!OUT_PART_VALID=part_valid.vw
!OUT_TEST=test.vw

LOG = True

In [3]:
# remove html tags

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ' '.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [4]:
train_target = pd.read_csv(os.path.join(PATH_TO_DATA, 
                                        'train_log1p_recommends.csv'), 
                           index_col='id')

In [5]:
# transform data to vw format
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

lmtzr = WordNetLemmatizer()
stop = set(stopwords.words("english") + list(string.punctuation) + ["’", "“", "”"])
tokenize = lambda s: " ".join(lmtzr.lemmatize(i) for i in word_tokenize(s.lower()) if i not in stop)


def to_vw(data, label):
    exclude = set(string.punctuation)
       
    content = strip_tags(data["content"]).replace("\n", " ").replace(":", "").replace("|", "")
    content = tokenize(content)
    
    title = strip_tags(data["title"]).replace("\n", " ").replace(":", "").replace("|", "")
    title = tokenize(title)
    
    author = strip_tags(data["meta_tags"]["author"]).replace("\n", " ").replace(":", "").replace("|", "")
    author = "".join(ch for ch in author if ch not in exclude).replace(" ", "_").lower()
    
    domain = data["domain"].replace("\n", " ").replace(":", "").replace("|", "").lower()
    
    # date time
    dt = datetime.strptime(data["published"]["$date"][:-1], "%Y-%m-%dT%H:%M:%S.%f")
    month = dt.month
    weekday = dt.weekday()
    hour = dt.hour 
    read_time = data["meta_tags"]["twitter:data1"].split(" ")[0]
    read_time = read_time if read_time else 0.1
    
    # binary
    is_weekend = 1 if weekday in (5, 6) else 0
    is_night = 1 if hour in range(0, 7) else 0
    is_morning = 1 if hour in range(7, 12) else 0
    is_noon = 1 if hour in range(12, 19) else 0
    is_eve = 1 if hour in range(19, 24) else 0
    is_image = 1 if data["image_url"] else 0


    out = [
        str(np.log(label)),
        "a_content %s" % content,
        "b_author %s" % author,
        "c_title %s" % title,
        "d_domain %s" % domain,
        "e_num 10:%s 11:%s 12:%s 13:%s" % (month, weekday + 1, hour + 1, read_time),
        "f_binary 0:%s 1:%s 2:%s 3:%s 4:%s 5:%s" % 
        (is_weekend, is_night, is_morning, is_noon, is_eve, is_image)
    ]
    
    return str(" |".join(out).encode('ascii', 'ignore').strip())[2:-1] + "\n"


In [7]:
import codecs

with codecs.open(os.path.join(PATH_TO_DATA, INP_TRAIN), encoding="utf-8") as inp_json, \
     codecs.open(os.path.join(PATH_TO_DATA, OUT_TRAIN), 'w', encoding="utf-8") as out_train:
    N = 52699
#     N = 100        
    header = False
    
    for n, json_ in enumerate(tqdm_notebook(inp_json, total=N)):
        if n == N: break

        json_data = json.loads(json_)
        label = train_target.iloc[n]["log_recommends"]
        out = to_vw(json_data, label)
        out_train.write(out)


In [8]:
# split train into part_train and part_valid

!split -l 36889 $PATH_TO_DATA/$OUT_TRAIN $PATH_TO_DATA/$OUT_TRAIN"_"

!mv $PATH_TO_DATA/$OUT_TRAIN"_aa" $PATH_TO_DATA/$OUT_PART_TRAIN
!mv $PATH_TO_DATA/$OUT_TRAIN"_ab" $PATH_TO_DATA/$OUT_PART_VALID

In [9]:
# from validation target file 

!cut -f 1 -d ' ' $PATH_TO_DATA/$OUT_PART_VALID > $PATH_TO_DATA/part_valid_target.txt

In [10]:
%%time

!vw -d $PATH_TO_DATA/$OUT_PART_TRAIN -k --cache_file=$PATH_TO_DATA/medium.cache \
  --loss_function squared --ngram=2 --passes 5 -b 24 -l 0.5 --power_t=0.5 --l2=1e-5 \
  -f $PATH_TO_DATA/model.vw #--readable_model=$PATH_TO_DATA/readable_model.vw

Generating 2-grams for all namespaces.
using l2 regularization = 1e-05
final_regressor = ../../data/arktur_medium//model.vw
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ../../data/arktur_medium//medium.cache
Reading datafile = ../../data/arktur_medium//part_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
3.735672 3.735672            1            1.0   1.9328   0.0000     1235
2.101581 0.467491            2            2.0   1.7891   1.1053      907
1.536927 0.972273            4            4.0   0.8340   0.2407      485
0.921641 0.306355            8            8.0   1.6958   1.9328     4389
0.789018 0.656394           16           16.0   0.3266   0.3900      799
0.559749 0.330479           32           32.0   1.3425   1.8813     1607
0.454237 0.348724           64           64.0   0.0940   0.2947

In [11]:
# predict for validation and MAE

!vw -i $PATH_TO_DATA/model.vw -t -d $PATH_TO_DATA/$OUT_PART_VALID \
-p $PATH_TO_DATA/part_valid_predictions.txt --quiet

from sklearn.metrics import mean_absolute_error

y_true = np.loadtxt(PATH_TO_DATA + '/part_valid_target.txt')
y_pred = np.loadtxt(PATH_TO_DATA + '/part_valid_predictions.txt')

mean_absolute_error(y_true, y_pred)

0.41619083061151058

In [12]:
%%time

!vw -d $PATH_TO_DATA/$OUT_TRAIN -k --cache_file=$PATH_TO_DATA/medium.cache \
  --loss_function squared --ngram=2 --passes 5 -b 24 -l 0.5 --power_t=0.5 --l2=1e-5 \
  -f $PATH_TO_DATA/model.vw #--readable_model=$PATH_TO_DATA/readable_model.vw

Generating 2-grams for all namespaces.
using l2 regularization = 1e-05
final_regressor = ../../data/arktur_medium//model.vw
Num weight bits = 24
learning rate = 0.5
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
creating cache_file = ../../data/arktur_medium//medium.cache
Reading datafile = ../../data/arktur_medium//train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
3.735672 3.735672            1            1.0   1.9328   0.0000     1235
2.101581 0.467491            2            2.0   1.7891   1.1053      907
1.536927 0.972273            4            4.0   0.8340   0.2407      485
0.921641 0.306355            8            8.0   1.6958   1.9328     4389
0.789018 0.656394           16           16.0   0.3266   0.3900      799
0.559749 0.330479           32           32.0   1.3425   1.8813     1607
0.454237 0.348724           64           64.0   0.0940   0.2947     

In [17]:
# create test data

import codecs

with codecs.open(os.path.join(PATH_TO_DATA, INP_TEST), encoding="utf-8") as inp_json, \
     codecs.open(os.path.join(PATH_TO_DATA, OUT_TEST), 'w', encoding="utf-8") as out_test:
    N = 39492        

    for n, json_ in enumerate(tqdm_notebook(inp_json, total=N)):
        if n == N: break

        json_data = json.loads(json_)

        out = to_vw(json_data, 1)
        out_test.write(out)

In [18]:
# predict for test

!vw -i $PATH_TO_DATA/model.vw -t -d $PATH_TO_DATA/test.vw \
-p $PATH_TO_DATA/test_predictions.txt --quiet

In [19]:
# create submission

submission = pd.read_csv(PATH_TO_DATA + '/sample_submission.csv', index_col='id')
if LOG:
    submission['log_recommends'] = np.exp(np.loadtxt(PATH_TO_DATA + '/test_predictions.txt'))
else:
    submission['log_recommends'] = np.loadtxt(PATH_TO_DATA + '/test_predictions.txt')
submission.to_csv(PATH_TO_DATA + '/submit/best_n2_p5_b24_l05_p05_l2_1e5.csv')